In [ ]:
import sys
import random

sys.path.append('../')
import util
from test_framework.key import SECP256K1_ORDER, SECP256K1_FIELD_SIZE, ECKey, ECPubKey

# 0.2 Elliptic Curve Math (Review)


Elliptic Curve math involves scalars and Points. 
* A scalar is a positive integer which is smaller than secp256k1 the field size. 
* A point `P(x,y)` lies on the secp256k1 curve.

The following provides an overview of all operations of scalars and points over elliptic curves. In Bitcoin, key pair generation and signing is performed over the SECP256k1 curve.

![test](../images/ec_math0.jpg)



### Classes / Methods for Elliptic Curve Math

**`Integers`:** All Scalar operations over SECP251k1 can be performed with python integers and the modulo `%` operator. 

Scalar addition, subtraction, multiplication and division over SECP256k1 are modulo a large prime number SECP256K1_ORDER. 


* All scalar operations are performed modulo n (`SECP256K1_ORDER`).
* Addition: `a + b % SECP256K1_ORDER`
* Subtraction: `-a = SECP256K1_ORDER - a`
* Multiplication: `a * b % SECP256K1_ORDER`
* Division (Fermat's little Theorem): `1/b = b ** (SECP256K1_ORDER-2) % SECP256K1_ORDER`

**`ECKey`:** The Bitcoin Core library provides a private key class which can also perform certain scalar operations.
* Addition: `a + b`
* Subtraction: 
    * `b.negate()`
    * `a + b` 
* Multiplication: `a * b`
* Division: `a * 1/b` (See Fermat's little theorem) 

![test](../images/ec_math1.jpg)

#### 1.1 Example: Scalar Addition over SECP256K1 order

Addition can be performed with integer modulo math in python or with the private key class `ECKey`. We can set a `ECKey` object to a certain value, or generate a new private key with the `generate` method.

In the example below, addition is performed with both integers and the `ECKey` class, and evaluated for equality.

In [ ]:
# scalar operations.
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)
ab = (a + b) % SECP256K1_ORDER

# ECkey(operation)
a_key = ECKey()
b_key = ECKey()
a_key.set(a.to_bytes(32,'big'), True) 
b_key.set(b.to_bytes(32,'big'), True) 
ab_key = a_key + b_key

# Ensure equivalency.
print(int().from_bytes(ab_key.get_bytes(),'big') == ab)


#### 1.2 Example: Scalar Multiplication over SECP256K1 order

In the example below, multiplication is performed with both integers and the `ECKey` class, and evaluated for equality.

In [ ]:
# Integer operations
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)
ab = (a * b) % SECP256K1_ORDER

# ECkey(operation)
a_key = ECKey()
b_key = ECKey()
a_key.set(a.to_bytes(32,'big'), True) 
b_key.set(b.to_bytes(32,'big'), True) 
ab_key = a_key * b_key

# Ensure operations are equivalent.
print(int().from_bytes(ab_key.get_bytes(),'big') == ab)


#### 1.3 _Programming Exercise:_ Commutative property of scalar operations

In this exercise we wish to demonstrate the commutative property of scalar addition and multiplication, whilst getting familiarized with both integer modulo operations and the private key `ECKey` methods.

Consider: 
* `a + b = b + a` over SECP256k1
* `a * b = b * a` over SECP256k1

Notes:
* Demonstrate that both equations hold.
* Compute the left sides with the Python `int` class.
* Compute the right sides with the Bitcoin Core `ECKey` class.


In [ ]:
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)

# Left: Compute a + b
ab = (a + b) % SECP256K1_ORDER

# Right: Compute b + a
a_key = ECKey()
b_key = ECKey()
a_key.set(a.to_bytes(32,'big'), True) 
b_key.set(b.to_bytes(32,'big'), True) 
ba_key = b_key + a_key

# Left/Right: Assert equality
print("Commutative Add:", ab == int().from_bytes(ba_key.get_bytes(),'big'))

# Left: Compute a * b
ab = (a * b) % SECP256K1_ORDER

# Right: Compute b * a
ba_key = b_key * a_key

# Left/Right: Assert equality
print("Commutative Mult:", ab == int().from_bytes(ba_key.get_bytes(),'big'))


#### 1.4 _Programming Exercise:_ Distributivity of scalar operations

In this exercise we wish to demonstrate the distributivity property of scalar addition and multiplication.

Consider: `(a - b) * c = a * c - b * c` over SECP256k1
* Demonstrate that the equation holds
* Compute the left side with the Python `int` class.
* Compute the right side with the Bitcoin Core `ECKey` class.


In [ ]:
a = random.randrange(1, SECP256K1_ORDER)
b = random.randrange(1, SECP256K1_ORDER)
c = random.randrange(1, SECP256K1_ORDER)

# Left: Compute (a - b) * c
ab = (a + (SECP256K1_ORDER - b)) % SECP256K1_ORDER
ab_c = (ab * c) % SECP256K1_ORDER

a_key = ECKey()
b_key = ECKey()
c_key = ECKey()

# Right: Compute -b * c
# Negate a ECKey with negate() method.
a_key.set(a.to_bytes(32,'big'), True)
b_key.set(b.to_bytes(32,'big'), True)
b_key.negate()
c_key.set(c.to_bytes(32,'big'), True)

# Right: Compute a * c
ac_key = a_key * c_key
bc_key = b_key * c_key

# Right: Compute a * c - b * c
ac_bc_key = ac_key + bc_key

# Left/Right: Assert equality
print("Distributivity Add/Mult:", ab_c.to_bytes(32,'big') == ac_bc_key.get_bytes())


#### 1.5 Example: Point Addition over SECP256k1

The public key `ECPubkey` class can be derived from `ECKey` with the `ECKey.get_pubkey` method. 

In the following example, we perform point addition with `ECPubkey.__add__`.


In [ ]:
a = ECKey()
b = ECKey()
a.generate()
b.generate()

A = a_key.get_pubkey()
B = b_key.get_pubkey()
AB = A + B

print(AB.get_bytes().hex())

#### 1.6  _Programming Exercise:_ Distributivity over scalar and ec points.

In this exercise we wish to demonstrate the distributivity property of scalar/point operations, whilst getting familiarized with both integer modulo operations and the public key `ECPubKey` methods.

Consider: `(a - b) * C = a * C - b * C`


In [ ]:
a = ECKey()
b = ECKey()
a.generate()
b.generate()

# Left: Compute a - b
b.negate()
a_neg_b = a + b

# Left: Compute (a - b) * G   
L = a_neg_b.get_pubkey()

# Right: Compute aG     
a_pk = a.get_pubkey()

# Right: Compute aG - bG
b_neg_pk = b.get_pubkey()
R = a_pk + b_neg_pk

# Left/Right: Assert equality
print(L.get_bytes() == R.get_bytes())
